In [1]:
import reltest
import reltest.util as util
from reltest.mctest import MCTestPSI, MCTestCorr
from reltest.mmd import MMD_U, med_heuristic
from reltest import kernel
from helper import download_to, summary, fid

import numpy as np
import os
import logging
import pandas as pd
logging.getLogger().setLevel(logging.INFO)
%load_ext autoreload
%autoreload 2

#### Download Dataset (Default = ```False```)

To download dataset change ```download``` to ```True```.

In [2]:
download = False

feature_url = "http://ftp.tuebingen.mpg.de/pub/is/wittawat/kmod_share/problems/celeba/inception_features/"
feature_dir = os.getcwd() + "/celeba_features"
celeba_classes = ['gen_smile', 'gen_nonsmile', 'ref_smile', 'ref_nonsmile']
mix_ratio = [1,1,0,0]
inception_dim = 2048
## Normalised ratio
celeb_class_ratio = dict(zip(celeba_classes, mix_ratio/np.sum(mix_ratio)))

if download:
    os.mkdir(feature_dir)
    for celeba_class in  celeba_classes:
        filename= '{}.npy'.format(celeba_class)
        npy_path = os.path.join(feature_dir, filename)
        url_path = os.path.join(feature_url, filename)
        download_to(url_path,npy_path)

### Problem Setting

In [3]:
class CelebASampler():
    def __init__(self, model_classes_mix, ref_classes_mix):
        celeba_features = []
        for celeba_class in  celeba_classes:
            filename= '{}.npy'.format(celeba_class)
            npy_path = os.path.join(feature_dir, filename)
            celeba_features.append(np.load(npy_path))
        self.celeba_features =  dict(zip(celeba_classes, celeba_features))
        self.model_classes_mix = model_classes_mix
        self.ref_classes_mix = ref_classes_mix
    
    def sample(self, n_samples, seed):
        ## DISJOINT SET
        model_features = {}
        ref_samples = []
        with util.NumpySeedContext(seed=seed):
            ## FOR EACH CELEBA CLASS
            for key, features in self.celeba_features.items():
                # CALCULATE HOW MUCH SHOULD BE IN THE REFERENCE POOL
                n_ref_samples = int(np.round(self.ref_classes_mix[key] * n_samples))
                random_features = np.random.permutation(features)
                
                ## FOR THE CANDIDATE MODELS
                model_features[key] = random_features[n_ref_samples:]
                ## FOR THE REFERENCE
                ref_samples.append(random_features[:n_ref_samples])
        
        ## samples for models
        model_samples = []
        for j,class_ratios in enumerate(model_classes_mix):
            model_class_samples = []
            for i, data_class in enumerate(class_ratios.keys()):
                n_class_samples = int(np.round(class_ratios[data_class] * n_samples))
                seed_class = i*n_samples+seed*j
                with util.NumpySeedContext(seed=seed_class):
                    indices = np.random.choice(model_features[data_class].shape[0], n_class_samples)
                model_class_samples.append(model_features[data_class][indices])
            class_samples = dict(zip(class_ratios.keys(),model_class_samples))
            model_class_stack = np.vstack(list(class_samples.values()))
            model_samples.append(model_class_stack)
            assert model_class_stack.shape[0] == n_samples, "Sample size mismatch: {0} instead of {1}".format(samples.shape[0],n)
            
        with util.NumpySeedContext(seed=seed+5):
            ref_samples = np.random.permutation(np.vstack(ref_samples))
            model_samples = [np.random.permutation(samples) for samples in model_samples]
        assert ref_samples.shape[0] == n_samples, "Sample size mismatch: {0} instead of {1}".format(samples.shape[0],n)
        return model_samples, ref_samples

In [4]:
model_mixtures = [
    [0.50,0.50,0.0,0.0], # Model 5
    [0.0,0.0,0.60,0.40], # Model 2
    [0.0,0.0,0.40,0.60], # Model 3
    [0.0,0.0,0.51,0.49], # Model 4
    [0.0,0.0,0.52,0.48], # Model 1
]
ref_mixtures = [[0,0,0.5,0.5]]
def class_mix_ratio(classes, mixtures):
    return [dict(zip(classes, mixture/np.sum(mixture))) for mixture in mixtures]
model_classes_mix = class_mix_ratio(celeba_classes, model_mixtures)
ref_classes_mix = class_mix_ratio(celeba_classes, ref_mixtures)[0]
n_samples = 2000
n_models = len(model_mixtures)
setting = {'n': n_models,
           'dim':inception_dim}
celeba = CelebASampler(model_classes_mix,ref_classes_mix)

In [5]:
n_trials = 100
psi_res = []
cor_res = []
fid_scores = []
for seed in range(n_trials):
    ## Sample from model
    model_samples, ref_samples = celeba.sample(n_samples, seed)
    
    psiTest = MCTestPSI(ref_samples)
    corTest = MCTestCorr(ref_samples)
    
    ## Perform Test
    bandwidth = med_heuristic(model_samples, ref_samples,
                subsample=1000)
    mmd_u = MMD_U(kernel.KIMQ())
    psi_res.append(psiTest.perform_tests(model_samples,mmd_u))
    cor_res.append(corTest.perform_tests(model_samples, mmd_u, split=0.5, density=False, correction=0))
    
    ## Calculate FID
    fid_score = []
    for j, model_samples in enumerate(model_samples):
        fid_score.append(fid(model_samples,ref_samples))
    fid_scores.append(fid_score)

/Users/jenninglim/Documents/projects/model-comparison-test/reproduce-results/helper.py:158: ComplexWarning: Casting complex values to real discards the imaginary part
  np.trace(cov_g) + np.trace(cov_r) - 2 * np.trace(covmean))


KeyboardInterrupt: 

In [6]:
n_models = len(model_mixtures)
print(summary(psi_res,n_models))
print(summary(cor_res,n_models))

{'av_rej': array([0.98913043, 0.36956522, 0.27173913, 0.02173913, 0.05434783]), 'av_sel': array([0.        , 0.01086957, 0.0326087 , 0.52173913, 0.43478261]), 'av_time': 13.940219404904742}
{'av_rej': array([1.        , 0.18478261, 0.19565217, 0.0326087 , 0.        ]), 'av_sel': array([0.        , 0.06521739, 0.10869565, 0.36956522, 0.45652174]), 'av_time': 3.629944656206214}


In [9]:
fid_results = { 'ind_sel':np.zeros(n_models),
          'av_fid': np.zeros(n_models)}
for trial in fid_scores:
    l_fids=[model1 for model1 in trial]
    ind_sel = np.argmin(l_fids)
    fid_results['ind_sel'][ind_sel] += 1/len(fid_scores)
    fid_results['av_fid'] =fid_results['av_fid']+ np.array(l_fids)/len(fid_scores)
fids=[]
for trial in fid_scores:
    fids.append([model1 for model1 in trial])
print(np.mean(np.array(fids),axis=0))
print(np.median(np.array(fids),axis=0))


[27.79131268 16.00136641 16.31978307 16.07260718 16.04231602]
[27.76755216 15.99755937 16.33001294 16.0470361  16.03376371]


In [10]:
dict_res = {
    'model_mix':model_mixtures,
    'test_psi_sel':summary(psi_res,n_models)['av_sel'],
    'test_psi_rej':summary(psi_res,n_models)['av_rej'],
    'test_cor_sel':summary(cor_res,n_models)['av_sel'],
    'test_cor_rej':summary(cor_res,n_models)['av_rej'],
    'fid_results_sel':fid_results['ind_sel'],
    'fid_results_av':fid_results['av_fid'],
    'fid_results_std':np.std(np.array(fids),axis=0),
}
pd.DataFrame.from_dict(dict_res)

,model_mix,test_psi_sel,test_psi_rej,test_cor_sel,test_cor_rej,fid_results_sel,fid_results_av,fid_results_std
0,"[0.5, 0.5, 0.0, 0.0]",0.000000,0.989130,0.000000,1.000000,0.000000,27.791313,0.586255
1,"[0.0, 0.0, 0.6, 0.4]",0.010870,0.369565,0.065217,0.184783,0.439560,16.001366,0.203531
2,"[0.0, 0.0, 0.4, 0.6]",0.032609,0.271739,0.108696,0.195652,0.043956,16.319783,0.228973
3,"[0.0, 0.0, 0.51, 0.49]",0.521739,0.021739,0.369565,0.032609,0.208791,16.072607,0.229746
4,"[0.0, 0.0, 0.52, 0.48]",0.434783,0.054348,0.456522,0.000000,0.307692,16.042316,0.214285
